#  INDEPTH LINEAR REGRESSION

## This is a wll detailed model in which we will we will explore indepth  a linear model regression model we will alter tune the model using the l1 and l2 parameters to improve our model .

## We will be using data from DATA.GOV The data is of Real estates sales from 2001-2020( this is a website where we use real time data thus giving more user experience on data that is actually real.  There are 14 columns in our dataset  we will need to get rid of the columns that we wonr use in our model  the columns include 1.Serial Number 2. List Year3.Date Recorded  4.Address  5. Town  6.Non use code 7. Assessor remarks 8. OpM remarks  we willthen proceed to preprocess the data 

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [84]:
df =pd.read_csv('Real_Estate_Sales_2001-2020_GL.csv')

C:\Users\stilinski\AppData\Local\Temp\ipykernel_11328\3797630619.py:1: DtypeWarning: Columns (0,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df =pd.read_csv('Real_Estate_Sales_2001-2020_GL.csv')


In [85]:
df.head()

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020348,2020.0,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,NaN,NaN,NaN,NaN,NaN
1,20002,2020.0,10/02/2020,Ashford,390 TURNPIKE RD,253000.0,430000.0,0.5883,Residential,Single Family,NaN,NaN,NaN,NaN
2,200212,2020.0,03/09/2021,Avon,5 CHESTNUT DRIVE,130400.0,179900.0,0.7248,Residential,Condo,NaN,NaN,NaN,NaN
3,200243,2020.0,04/13/2021,Avon,111 NORTHINGTON DRIVE,619290.0,890000.0,0.6958,Residential,Single Family,NaN,NaN,NaN,NaN
4,200377,2020.0,07/02/2021,Avon,70 FAR HILLS DRIVE,862330.0,1447500.0,0.5957,Residential,Single Family,NaN,NaN,NaN,NaN


In [86]:
df.columns

Index(['Serial Number', 'List Year', 'Date Recorded', 'Town', 'Address',
       'Assessed Value', 'Sale Amount', 'Sales Ratio', 'Property Type',
       'Residential Type', 'Non Use Code', 'Assessor Remarks', 'OPM remarks',
       'Location'],
      dtype='object')

In [87]:
df.shape

(601172, 14)

we will be checking for null values

In [88]:
df.isnull().sum().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of Serial Number            0
List Year                4
Date Recorded            6
Town                     4
Address                 52
Assessed Value           5
Sale Amount              5
Sales Ratio              5
Property Type       349568
Residential Type    355431
Non Use Code        444475
Assessor Remarks    550084
OPM remarks         599183
Location            469478
dtype: int64>

In [89]:
df.drop(['Non Use Code','Assessor Remarks','OPM remarks','Serial Number','List Year','Date Recorded','Location','Address','Town'],axis =1, inplace =True)

In [90]:
df['Property Type'].value_counts()

Single Family     132024
Residential        60728
Condo              37970
Two Family          9497
Three Family        4828
Vacant Land         3163
Commercial          1981
Four Family          694
Apartments           486
Industrial           228
Public Utility         5
Name: Property Type, dtype: int64

In [91]:
df['Property Type'].fillna('Single Family', inplace=True)

In [92]:
df['Residential Type'].value_counts()

Single Family    175428
Condo             50330
Two Family        12698
Three Family       6323
Four Family         962
Name: Residential Type, dtype: int64

In [93]:
df['Residential Type'].fillna('Single Family', inplace=True)

In [94]:
df.dropna(inplace=True)

## We will thus encode our data to change our non-numerical values to numerical values  this is to make our data easily understood for our model implementation

In [95]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column contains non-numeric values
    if df[column].dtype == object:
        # Encode the non-numeric values
        df[column] = label_encoder.fit_transform(df[column])

We will be standardizing the data to bring all our valuesin a certain range an then do a PCA (for dimensionality reduction(  we will continue to test_split our data anfd then we will  move to fit our linear regression model   wewill then later find our Mean Bsolute error.

In [130]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


# Step 3: Separate features and target
X = df.drop(columns=['Sale Amount'])
y = df['Sale Amount']

# Step 4: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 5: Perform PCA
pca = PCA(n_components=3)  # Retain all components
X_pca = pca.fit_transform(X_scaled)

# Step 6: Analyze explained variance
explained_variance_ratio = pca.explained_variance_ratio_

# Step 7: Analyze component loadings
component_loadings = pca.components_

# Step 8: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=2)

# Step 9: Regression analysis
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

# Step 10: Make predictions and calculate MAE
y_pred = regression_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)


Mean Absolute Error (MAE): 238892.2932487012


In [120]:
mean_absolute_error(y_test,y_pred)/y_test.mean()

0.6375822344833649

In [121]:
from xgboost import XGBRegressor
boost_model =XGBRegressor()
boost_model.fit(X_train,y_train)
boost_pred =boost_model.predict(X_test)
mean_absolute_error(y_test,boost_pred)

139933.68027819935

In [43]:
mean_absolute_error(y_test,boost_pred)/y_test.mean()

0.41104356561639205

## We will the proceed to do our lasso regression model and see whether our model perfomance changes as we ca see our model does not cahnge the perfomance .

In [135]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=5, max_iter=100, tol=0.1)
lasso_reg.fit(X_train, y_train)


Lasso(alpha=5, max_iter=100, tol=0.1)

In [136]:
boost_pred =lasso_reg.predict(X_test)
mean_absolute_error(y_test,boost_pred)

238892.91050585208

## We will then look at the ridge regression where we will also look at the perfomance and as we can see after l2 th perfomance actually improves 

In [123]:
from sklearn.linear_model import Ridge

Ridge_reg = Ridge(alpha=5, max_iter=1000, tol=0.01)
Ridge_reg.fit(X_train, y_train)

Ridge(alpha=5, max_iter=1000, tol=0.01)

In [122]:
boost_pred =Ridge_reg.predict(X_test)
mean_absolute_error(y_test,boost_pred)

235676.4389122